# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [3]:
df_products = data.groupby(['CustomerID','ProductName']).sum()
df_products.head()

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [4]:
pivot = pd.pivot_table(df_products, values='Quantity', index=['ProductName'], columns=['CustomerID'], fill_value=0)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [5]:
distances = pd.DataFrame(1/(1+squareform(pdist(pivot.T,'euclidean'))),index=pivot.columns,columns=pivot.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [6]:
distances = pd.DataFrame(1/(1+squareform(pdist(pivot.T,'euclidean'))),index=pivot.columns,columns=pivot.columns)
similarities = distances[33].sort_values(ascending=False)[1:].head(5)
similarities

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
records = df_products.loc[similarities.index]
records

SalesID  ProductID  Quantity
CustomerID ProductName                                                  
264        Apricots - Halves                3730301        278         1
           Apricots Fresh                   5035449        158         1
           Bacardi Breezer - Tropical        756802         49         1
           Bagel - Plain                    6728381        262         1
           Banana - Leaves                  6298459        271         1
           Bananas                          3387684        146         1
           Bandage - Flexible Neon          2009347        273         1
           Beef - Tenderlion, Center Cut    6635740        352         1
           Berry Brulee                     2399761        399         1
           Bread - Roll, Canadian Dinner    4370801        341         1
           Bread Foccacia Whole             3526383        378         1
           Cheese - Brie, Triple Creme      6213860         30         1
           Cheese - Parmesan Cubes          2229328         85         1
           Cheese Cloth No 100                24688        270         1
           Chocolate - Dark                 6793282        360         2
           Cocoa Butter                     4874854        206         1
           Coffee - Dark Roast              5402741        310         1
           Coffee Decaf Colombian           2585657        361         1
           Cookie Chocolate Chip With        928142          2         1
           Crab - Dungeness, Whole          3989265        291         1
           Crackers - Trio                  5975879        126         1
           Creme De Banane - Marie          1848293        371         1
           Cumin - Whole                    2830017        362         1
           Duck - Breast                    3825962        233         1
           Flour - Teff                     3883387        368         1
           Grouper - Fresh                  6651218        245         1
           Guinea Fowl                      9102273        348         2
           Hinge W Undercut                 6577594        297         1
           Langers - Ruby Red Grapfruit     1307257        150         1
           Lemonade - Natural, 591 Ml       3239196        282         1
...                                             ...        ...       ...
3535       Juice - Orange                   3338309        438         1
           Ketchup - Tomato                 5390738        117         1
           Lamb - Ground                    2957322         83         1
           Lamb - Pieces, Diced             4503728        417         1
           Milk - 2%                        6648211        168         1
           Milk Powder                       813539        353         1
           Muffin Batt - Blueberry Passion  5375388        121         1
           Ocean Spray - Kiwi Strawberry    5454259        113         1
           Ocean Spray - Ruby Red            622468        115         1
           Pasta - Orecchiette              3076090        108         1
           Pepper - Black, Whole            2594726        187         1
           Sausage - Breakfast              2242428         57         1
           Scallops - 10/20                 5949516        216         1
           Scallops 60/80 Iqf                108617        111         1
           Sea Bass - Whole                  155998        277         1
           Sherry - Dry                     1911737        172         1
           Sole - Dover, Whole, Fresh        999748        222         1
           Soup - Campbells Bean Medley     5211655        382         1
           Soupfoamcont12oz 112con          3229163        263         1
           Steam Pan - Half Size Deep       5567372        116         1
           Sugar - Fine                     3075019        209         1
           Table Cloth 54x72 White          5841357         36         1
           Tahini Paste      

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
similar = records.groupby(['ProductName']).sum().Quantity.sort_values(ascending=False)
similar = pd.DataFrame(similar)
similar.head()

,Quantity
ProductName,
Butter - Unsalted,3
Soup - Campbells Bean Medley,3
Towels - Paper / Kraft,3
Wine - Ej Gallo Sierra Valley,3
"Pepper - Black, Whole",3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
top5_products = similar[~similar.index.isin(df_products.loc[33].index)]
top5_products.head()

,Quantity
ProductName,
Butter - Unsalted,3
Soup - Campbells Bean Medley,3
Wine - Ej Gallo Sierra Valley,3
Wine - Blue Nun Qualitatswein,3
Scallops 60/80 Iqf,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [ ]:
dictionary = {}
customer_list = list(data.CustomerID)

for customer in customer_list:
    similarities = distances[customer].sort_values(ascending=False)[1:6]
    records = df_products.loc[similarities.index]
    similar = records.groupby(['ProductName']).sum().Quantity.sort_values(ascending=False)
    top5_products = similar[~similar.index.isin(df_products.loc[customer].index)].head()
    dictionary[customer] = top5_products.to_dict()
    if len(dictionary) == len(customer_list):
        break
    
for product in dictionary:
    dictionary[product] = [x for x in dictionary[product]]
    
#Por algún motivo que no consigo descrifrar, el código no termina de ejecutarse entonces lo he forzado.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
recommendations = pd.DataFrame(dictionary).T
recommendations = recommendations.rename(index=str, columns={0:'R1',1:'R2',2:'R3',3:'R4',4:'R5'})
recommendations.head()

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [ ]:
distances = pd.DataFrame(1/(1+squareform(pdist(pivot.T,'correlation'))),index=pivot.columns,columns=pivot.columns)

dictionary = {}
customer_list = list(data.CustomerID)

for customer in customer_list:
    similarities = distances[customer].sort_values(ascending=False)[1:6]
    records = df_products.loc[similarities.index]
    similar = records.groupby(['ProductName']).sum().Quantity.sort_values(ascending=False)
    top5_products = similar[~similar.index.isin(df_products.loc[customer].index)].head()
    dictionary[customer] = top5_products.to_dict()
    if len(dictionary) == len(customer_list):
        break
    
for product in dictionary:
    dictionary[product] = [x for x in dictionary[product]]
    
recommendations = pd.DataFrame(dictionary).T
recommendations = recommendations.rename(index=str, columns={0:'R1',1:'R2',2:'R3',3:'R4',4:'R5'})
recommendations.head()